### Importando bibliotecas 

In [1]:
import cvxpy as cp
import numpy as np
import mosek
import cplex



### Definindo problema de otimização: variáveis e constraints

In [2]:
vars = 2

# Instanciando o vetor de variáveis de decisão
x = cp.Variable(shape=vars, boolean=False)

# Criando as matrizes e vetores que defininem as restrições do problema
inequalityMatrix = np.array([[2, 1], [1, 3]])
inequalityVector = np.array([1, 1])

positivityMatrix = np.array([[1, 0], [0, 1]])
positivityVector = np.array([0, 0])

# Forma matricial das 3 funções objetivo do exercício
objFunction1 = np.array([1, 1])
objFunction2 = np.array([-1, -1])
objFunction3 = np.array([1, 0])

# Vetor de constraints para o cvxpy
constraints = [inequalityMatrix @ x >= inequalityVector, positivityMatrix @ x >= positivityVector]  # Example of inequality constraints



### Definindo função objetvio e resolvendo o problema

In [3]:
# Para cada uma das funções objetivo, instanciar um problem do cvxpy, resolvê-lo, e printar resultado

objective = cp.Minimize(objFunction1 @ x)
problem = cp.Problem(objective=objective, constraints=constraints)
print("Starting optimization run for f1 - CPLEX")
resultadoCplex = problem.solve(solver="CPLEX", verbose=False)
if(resultadoCplex != None):
    optimalPoint = x.value
    optimum = objFunction1 * x.value
    print("f1: Optimum of {} found at x = {}".format(resultadoCplex, optimalPoint))
else:
    print("Infeasible or unbounded problem.")

print("")
objective = cp.Minimize(objFunction2 @ x)
problem = cp.Problem(objective=objective, constraints=constraints)
print("Starting optimization run for f2 - CPLEX")
resultadoCplex = problem.solve(solver="CPLEX", verbose=False)
if(resultadoCplex != None):
    optimalPoint = x.value
    optimum = objFunction2 * x.value
    print("f2: Optimum of {} found at x = {}".format(resultadoCplex, optimalPoint))
else:
    print("Infeasible or unbounded problem.")

print("")
objective = cp.Minimize(objFunction3 @ x)
problem = cp.Problem(objective=objective, constraints=constraints)
print("Starting optimization run for f3 - CPLEX")
resultadoCplex = problem.solve(solver="CPLEX", verbose=False)
if(resultadoCplex != None):
    optimalPoint = x.value
    optimum = objFunction3 * x.value
    print("f3: Optimum of {} found at x = {}".format(resultadoCplex, optimalPoint))
else:
    print("Infeasible or unbounded problem.")

# note que nesse último caso, o solver nos repassa um ponto específico de mínimo com o menor valor numérico
# da variável livre x2. No entanto, qualquer ponto da semi-reta x2 >= 1 seria igualmente válido. 


Starting optimization run for f1 - CPLEX


f1: Optimum of 0.6 found at x = [0.4 0.2]

Starting optimization run for f2 - CPLEX
Infeasible or unbounded problem.

Starting optimization run for f3 - CPLEX


c:\Users\arthu\venvs\optmin\Lib\site-packages\cvxpy\problems\problem.py:1409: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


f3: Optimum of 0.0 found at x = [0. 1.]


## Programação Semi-Definida:

In [8]:
# No caso da programção semi-definida, seguimos os mesmo passos do exercício anterior
# Aqui no entanto, nossas variáveis serão componentes de uma matriz simétrica

# código feio horroroso para construir as matrizes do problema
A = np.zeros((4,4))
B = np.zeros((4,4))
Id = np.identity(4)

A[0,3] = A[1,2] = A[2,3] = A[3,0] = B[0,0] = B[3,3] = 1
B[1,1] = B[2,2] = -1

# instanciando uma matriz simétrica como variável de decisão
X = cp.Variable((4,4), symmetric=True)

# criando lista de constraints para o cvxpy
constraints = [cp.trace(B @ X) == 1, cp.trace(X) == 1, X >> 0]

# instanciando e resolvendo problema 
objective = cp.Maximize(cp.trace(A @ X))
prob = cp.Problem(objective=objective, constraints=constraints)
prob.solve(solver="MOSEK")

# mostrando resultados
# truncamos o resultado final pois haviam pequenos devios nos valores, muito provavelmente advindos de erros de ponto flutuante
print("The optimal value is", '%.3f'%(prob.value))
print("A solution X is")
print(X.value.round(3))

The optimal value is 1.000
A solution X is
[[ 0.5  0.   0.   0.5]
 [ 0.  -0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.5  0.   0.   0.5]]
